In [2]:
import pandas as pd
import scipy.stats as st
import numpy as np
pd.set_option('display.max_columns', None)

----
1) Видвинем следуюущую гипотезу - отрицательно ли влияет наличие подписки на результата звонка оператора?

Будем проверять с помощью критерия Хи-квадрат. Построим таблицу сопряженности, где стобцами будут 0 и 1 - признаки успешности или неуспешности звонка, а строками тоже 0 и 1 - признак есть подписка Про или нет ее

In [4]:
tbank_df = pd.read_csv('new_df.csv')
tbank_df.head()

,customer_id,age,gender_cd,region_size,foreign_flg,entrepreneur_flg,auto_flg,traveler_flg,partnership_flg,child_amt,realty_type,segment_cd,bundle_nm,confirmed_income,predicted_income,product_cd,functional_cd,finish_dttm,state_talk_time_sec,wo_hit_status_result_desc,wo_hit_status_reason_desc,employee_id,emp_age,emp_gender_cd,emp_children_cnt,emp_work_months_cnt,emp_citizenship_country_cd,emp_marital_status_cd,result_calling,product,age_group,age_difference,functional,with_pro
0,348648289,44.0,1.0,село,0.0,0.0,0.0,0.0,1.0,1.0,1.0,r_01 (6),Pro,NaN,NaN,Cash Loan,Upsell,2023-10-25 20:00:00,151.0,"Дозвон, Отказ",Другое,7004.0,36,W,NaN,2,RUS,NaN,0,Кредит,36-50,8.0,Предложения и продажи,1
1,345496995,23.0,0.0,город,0.0,0.0,1.0,0.0,1.0,0.0,0.0,t_05 (19),Без подписки,NaN,NaN,MVNO,Afterfilling,2023-06-16 07:00:00,68.0,"Дозвон, Успешно",Назначена встреча,80760.0,52,W,NaN,6,NaN,NaN,1,Другое,14-25,29.0,Другое,0
2,64946855,31.0,1.0,город,0.0,0.0,0.0,0.0,1.0,0.0,1.0,t_07 (29),Без подписки,NaN,15362.87,Cash Loan,Preapprove,2023-11-24 07:00:00,36.0,"Дозвон, Отказ",Отказ прослушать предложение,184241.0,43,W,NaN,6,RUS,UNM,0,Кредит,26-35,12.0,Предложения и продажи,0
3,516320897,34.0,0.0,город,0.0,0.0,0.0,0.0,1.0,0.0,0.0,t_10 (40),Без подписки,NaN,13233.30,Mortgage Insurance,Cold,2024-08-17 12:00:00,38.0,"Дозвон, Отказ",Страхуется в своей СК,106508.0,74,M,NaN,1,RUS,NaN,0,Страхование,26-35,40.0,Предложения и продажи,0
4,148619460,67.0,0.0,город,0.0,0.0,0.0,0.0,1.0,0.0,1.0,t_08 (30),Без подписки,NaN,NaN,Credit Cards,Preapprove,2022-07-27 14:00:00,24.0,"Дозвон, Отказ",Отказ прослушать предложение,104002.0,46,W,NaN,10,NaN,NaN,0,Кредитная карта,65+,21.0,Предложения и продажи,0


In [5]:
table = pd.crosstab(tbank_df['with_pro'], tbank_df['result_calling'])
table

result_calling,0,1
with_pro,,
0,5300,591
1,2867,208


In [6]:
chi2, p_value, dof, expected = st.chi2_contingency(table, correction=True)
print(f'Хи-квадрат значение - {chi2}')
print(f'P-value - {p_value}')
print(f'Доля успешных звонков без подписки - {int(np.round(591 / 5891, 2) * 100)}%')
print(f'Доля успешных звонков с подпиской Про - {int(np.round(208 / (208 + 2867), 2) * 100)}%')

Хи-квадрат значение - 26.181329515793152
P-value - 3.1081316540647736e-07
Доля успешных звонков без подписки - 10%
Доля успешных звонков с подпиской Про - 7%


Делаем выводы:
- У нас есть сильная связь между признаками, так как значение критерия 26
- Доля успешных звонков без подписки выше, чем с подпиской, несмотря на то, что клиенты с подпиской точно лучше знакомы с продуктами банка
- Мы можем отвергнуть гипотезу, так как p-value получился очень маленьким
- Так же можно добавить, что наш датафрейм обработан и количество данных в обработанном датафрейме в 2 раза меньше. Но если даже мы будем применять хи-квадрат и на больших данных, выводы не изменятся

----
2. Следующая гипотеза - успешность звонка у разных возрастных групп. Посчитаем самые большие группы и возьмем 2 из них

In [22]:
tbank_df['age_group'].value_counts()

age_group
36-50      3432
26-35      2533
14-25      1777
51-65      1090
65+         111
unknown      23
Name: count, dtype: int64

Логично, что возрастные группы 26-35 и 36-50 - самые большие. Поэтому будем смотреть, как влияет группа на успешность звонка (а если быть точным - в какой группе лучше всего проводить звонки?)

In [27]:
table_age = pd.crosstab(tbank_df['result_calling'], tbank_df['age_group'])
table_age.drop(['unknown', '14-25', '51-65', '65+'], axis=1, inplace=True)
print(f'Доля успешных звонков в группе 26-35 - {int(np.round(191 / (191 + 2342), 2) * 100)}%')
print(f'Доля успешных звонков в группе 36-50 - {int(np.round(316 / (316 + 3116), 2) * 100)}%')
table_age

Доля успешных звонков в группе 26-35 - 8%
Доля успешных звонков в группе 36-50 - 9%


age_group,26-35,36-50
result_calling,,
0,2342,3116
1,191,316


chi2, p_value, dof, expected = st.chi2_contingency(table_age, correction=True)
print(f'Хи-квадрат значение - {chi2}')
print(f'P-value - {p_value}')

Получилось p-value ниже 0.05, поэтому можно утверждать что есть статистически значимые различия между группами, но так как значение критерия довольно низкое, поэтому можно утверждать, что различия на самом деле не большие

**Вывод:** нет большого смысла делать больше звонков той или иной возрастной группе, так как статистически они слабо различаются

-----
3. Средняя зарплата у клиентов с Про и без подписки

In [95]:
median_income_1 = tbank_df.query('bundle_nm == "Pro"')['confirmed_income'].median()
with_pro = np.array(tbank_df.query('bundle_nm == "Pro"')['confirmed_income'].fillna(median_income_1))
median_income_2 = tbank_df.query('bundle_nm != "Pro"')['confirmed_income'].median()
no_pro = np.array(tbank_df.query('bundle_nm != "Pro"')['confirmed_income'].fillna(median_income_2))
print(f'Средняя зарплата с подпиской Pro = {np.round(with_pro.mean(), 2)}, без подписки = {np.round(no_pro.mean(), 2)}')
print(f'Значение стаистик  Колмогорова-Смирнова = {st.kstest(with_pro, no_pro).statistic}')
print(f'Значение p-value = {st.kstest(with_pro, no_pro).pvalue}')

Средняя зарплата с подпиской Pro = 14276.24, без подписки = 12140.76
Значение стаистик  Колмогорова-Смирнова = 0.9087122287960275
Значение p-value = 0.0


**Выводы:**
- распределения у этих выборок различаются
- если будем смотреть на среднее, то с подпиской у людей доход выше, чем без подписки
- статистически пункт выше подтверждается тем, что значение критерия довольно высокое (0.9), значит у нас большие различия между выборками

----
Из этих критериев и гипотез берем только одну - самую первую, которая должна помочь нам увеличить количество успешных звонков, особенно для клиентов с подпиской. Хоть и подписка Pro отрицательно влияет на результат звонка, у клиентов с подпиской доход выше, значит он может принести больше прибыли компании. Решение этой проблемы: 
1) Распределить более опытных операторов между людьмт с подпиской Про
2) Сделать рассылку в виде уведомлений, смс-сообщений, писем на почту с возможностью обратной связи - позвонить в банк
3) Персонализировать предложение под каждого клиента, чтобы вероятность успешного звонка была выше
4) Внедрить бонусы за полученное предложение от оператора
5) Обучить операторов работать с клиентами с Про-подпиской (Премиум не берем - у них премиум поддержка и эти люди хорошо знакомы с продуктами банка)

Здесь долю определить невозможно, так как у нас идут звонки операторов Тбанка своим клиентам